In [126]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader 
import torch.nn.functional as F 
from torch.autograd import Variable
import torch.optim as optim
from tqdm import tqdm
from torchsummary import summary
from torchvision import datasets, transforms
import time
import os

In [127]:
df = pd.read_csv('breast-cancer-wisconsin.data', header=None)
#             names=['code_number',
#               'clumb_thickness',
#               'cell_size_uniformity', 
#               'cell_shape_uniformity',
#               'marginal_adhesion',
#              'epithelial_sell_size', 
#              'bare_nuclei', 
#              'bland_chromatin',
#               'normal_nuleoli'
#              'mitoses',
#              'class'])


In [128]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [129]:
df.describe()

,0,1,2,3,4,5,7,8,9,10
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [130]:
df.shape

(699, 11)

In [131]:
np_arr = df.to_numpy()

In [132]:
X_replace = df.replace('?',0)

In [133]:
np_array = np.array(X_replace).astype(float)

In [134]:
df_tensor = torch.tensor(np_array)

In [135]:
class Breast_Cancer_Classifier(nn.Module):
    def __init__(self):
        super(Breast_Cancer_Classifier, self).__init__()

        self.linear_a = nn.Linear(11, 64)
        self.linear_b = nn.Linear(64, 1)

    def forward(self, x):
        x = self.linear_a(x)
        x = F.relu(x)
        y_pred = self.linear(x)
        return y_pred

model = Breast_Cancer_Classifier()

In [136]:
class BreastCancerDataset(Dataset):

    def __init__(self):  #Initialise the data, download etc
        df = pd.read_csv('breast-cancer-wisconsin.data', header=None)
        X_replace = df.replace('?',0)
        np_array = np.array(X_replace).astype(float)
        self.len = df.shape[0]
        self.x_data = torch.from_numpy(np_array[:, :10])
        self.y_data = torch.from_numpy(np_array[:, 10])

    def __getitem__(self, index):  #return one item on the index
        return self.x_data[index], self.y_data[index]

    def __len__(self): #return the data length
        return self.len

dataset = BreastCancerDataset()
train_loader = DataLoader(dataset=dataset, 
                            batch_size=32, 
                            shuffle=True, 
                            num_workers=0)
# help('torch.utils.data.DataLoader')

In [137]:
criterion = torch.nn.MSELoss(size_average=False)
optimizer = optim.SGD(model.parameters(), lr=0.01)
model.train()

# for epoch in range(2):
#     for i, data in enumerate(train_loader, 0):
#         inputs, labels = data
#         inputs, labels = Variable(inputs), Variable(labels)
#         y_pred = model(inputs) # Forward pass
    
#         # Compute Loss
#         loss = criterion(y_pred, labels)
#         print(epoch, i, loss.data[0])
        
#         # zero gradients, perform a backward pass and update the weights
        
#         # Backward pass
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
        
start_time = time.time()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # forward + backward + optimize
        outputs = model(inputs.float())   #is the same is model.forwward

        # computing the loss
        loss = criterion(outputs, labels)
        # back propagation 
        loss.backward()
        optimiser.step() #update parameters based on loss

        #  reset the gradients to zero before moving forward, because PyTorch accumulates gradients.
        optimiser.zero_grad()

        # print statistics
        running_loss += loss.item()
        if i % 64 == 0:    # print every 64 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

end_time = time.time()
print('Finished Training')
print('Training Time: ', end_time - start_time)

RuntimeError: size mismatch, m1: [32 x 10], m2: [11 x 64] at C:\w\1\s\tmp_conda_3.6_081743\conda\conda-bld\pytorch_1572941935551\work\aten\src\TH/generic/THTensorMath.cpp:197